In [1]:
import pandas as pd
import requests


# 3. Библиотека requests

In [2]:
url = 'https://www.cbr-xml-daily.ru/daily_json.js' # Определяем значение URL страницы для запроса
response = requests.get(url) # Делаем GET-запрос к ресурсу и результат ответа сохраняем в переменной response

In [3]:
response # Выводим значение response на экран как объект

<Response [200]>

In [4]:
print(response.status_code) # Выводим числовое значение response на экран

200


In [5]:
#  Задание 3.1
# Вы уже импортировали модуль requests в ваш код. Напишите строку кода, 
# при выполнении которой будет сделан GET-запрос к ресурсу https://www.cbr-xml-daily.ru/daily.xml, 
# а результат ответа будет сохранён в переменной response:
    
# response = requests.get(url = "https://www.cbr-xml-daily.ru/daily.xml") 

### <center> РАБОТАЕМ С ОТВЕТОМ

Текст ответа хранится в атрибуте text. Выведем значение атрибута на экран и посмотрим на его содержимое:

In [6]:
print(response.text) # Выводим содержимое атрибута text переменной response на экран

{
    "Date": "2022-04-23T11:30:00+03:00",
    "PreviousDate": "2022-04-22T11:30:00+03:00",
    "PreviousURL": "\/\/www.cbr-xml-daily.ru\/archive\/2022\/04\/22\/daily_json.js",
    "Timestamp": "2022-04-24T11:00:00+03:00",
    "Valute": {
        "AUD": {
            "ID": "R01010",
            "NumCode": "036",
            "CharCode": "AUD",
            "Nominal": 1,
            "Name": "Австралийский доллар",
            "Value": 53.9159,
            "Previous": 55.8293
        },
        "AZN": {
            "ID": "R01020A",
            "NumCode": "944",
            "CharCode": "AZN",
            "Nominal": 1,
            "Name": "Азербайджанский манат",
            "Value": 43.2382,
            "Previous": 44.1171
        },
        "GBP": {
            "ID": "R01035",
            "NumCode": "826",
            "CharCode": "GBP",
            "Nominal": 1,
            "Name": "Фунт стерлингов Соединенного королевства",
            "Value": 95.7844,
            "Previous": 97.8587
   

Как правило, при работе над реальным проектом на этапе получения данных мы уже понимаем, с какими форматами данных нам придётся работать. На предлагаемом для работы ресурсе информация есть как в JSON-формате, так и в XML. По нашему запросу ресурс возвращает информацию в JSON-формате, однако в настоящий момент результат хранится как единая строка. Проверить тип данных полученного ответа можно, воспользовавшись функцией type().

Для того чтобы удобно было работать с полученной информацией, нам необходимо преобразовать строку в словарь. В объект ответа Response  из библиотеки requests уже встроен метод json() .

Импортируем функцию pprint(), применим к полученному ответу метод json() и выведем полученный результат на экран:

In [7]:
from pprint import pprint # Импортируем функцию pprint()
currencies = response.json() # Применяем метод json()
pprint(currencies) # Выводим результат на экран)

{'Date': '2022-04-23T11:30:00+03:00',
 'PreviousDate': '2022-04-22T11:30:00+03:00',
 'PreviousURL': '//www.cbr-xml-daily.ru/archive/2022/04/22/daily_json.js',
 'Timestamp': '2022-04-24T11:00:00+03:00',
 'Valute': {'AMD': {'CharCode': 'AMD',
                    'ID': 'R01060',
                    'Name': 'Армянских драмов',
                    'Nominal': 100,
                    'NumCode': '051',
                    'Previous': 16.0333,
                    'Value': 15.7304},
            'AUD': {'CharCode': 'AUD',
                    'ID': 'R01010',
                    'Name': 'Австралийский доллар',
                    'Nominal': 1,
                    'NumCode': '036',
                    'Previous': 55.8293,
                    'Value': 53.9159},
            'AZN': {'CharCode': 'AZN',
                    'ID': 'R01020A',
                    'Name': 'Азербайджанский манат',
                    'Nominal': 1,
                    'NumCode': '944',
                    'Previous': 44.1171,


Например, по ключу Valute мы можем обратиться к вложенному словарю, который содержит информацию о мировых валютах. Выведем на экран, например, информацию о евро (EUR):

In [20]:
pprint(currencies['Valute']['EUR']) # Выводим на экран информацию о валюте евро

{'CharCode': 'EUR',
 'ID': 'R01239',
 'Name': 'Евро',
 'Nominal': 1,
 'NumCode': '978',
 'Previous': 81.2239,
 'Value': 80.0249}


In [15]:
# Задание 3.2
print(currencies['Valute']['CZK']['Name'])

Чешских крон


### <center> 4. Парсинг сайтов

✍ Ресурс, с которым мы работали в предыдущем юните, возвращал ответ о текущем курсе валют в удобном, структурированном формате, из которого было легко извлечь необходимую информацию. Довольно часто для получения информации приходится обращаться напрямую к HTML-страницам.

Для примера рассмотрим страницу, содержащую [статью](https://nplus1.ru/news/2021/10/11/econobel2021) с информацией о присуждении Нобелевской премии по экономике в 2021 году, и попробуем извлечь из неё заголовок статьи, опубликованной на странице, дату публикации, а также текст статьи.

### ОСНОВЫ HTML

Изучение языка HTML находится вне рамок этого курса, но для того, чтобы собирать информацию с веб-страниц, нет необходимости хорошо знать HTML. Достаточно понимать, что:

существуют теги с разными именами;
у тегов бывают атрибуты, такие как class и id;
теги образуют иерархическую структуру, то есть одни теги вложены в другие.

ДОПОЛНИТЕЛЬНО

Вы можете ознакомиться с информацией о HTML в справочнике, перейдя по [ссылке](http://htmlbook.ru/html).

### <center> ПОЛУЧАЕМ СОДЕРЖИМОЕ ВЕБ-СТРАНИЦЫ

Для этого отправим GET-запрос с помощью библиотеки requests и метода get() и посмотрим на текст ответа на наш запрос (как мы помним, он содержится в атрибуте text):

In [13]:
url = 'https://nplus1.ru/news/2021/10/11/econobel2021' # Определяем адрес страницы
response = requests.get(url)  # Выполняем GET-запрос
print(response.text)  # Выводим содержимое атрибута text

<!doctype html>
<html class="old-town-road no-js bg-fixed _no-bg" style="background-image:url(https://nplus1.ru/images/2021/10/11/9c6bcb5f7c01c6d88d49410e16ebc003.jpg)" lang="">
<head>
    
    <meta charset="utf-8">
    <meta http-equiv="x-ua-compatible" content="ie=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">
        <meta name="apple-mobile-web-app-capable" content="yes">
    <meta name="apple-mobile-web-app-status-bar-style" content="black">
                <meta name='wmail-verification' content='7991d7eb02d759f05b9050e111a7e3eb' />

    <link rel="canonical" href="https://nplus1.ru/news/2021/10/11/econobel2021" />

    <link rel="icon" href="/i/favicon-bw.png" />


        <title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>

    	    <meta itemprop="datePublished" content="2021-10-11"/>
	
	    <meta name="mediator_author" content="Илья Ферапонтов"/> 
	
        <!-- amp page -->
    <

### <center> 5. Библиотека BeautifulSoup